In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
### YOUR CODE HERE
# Figure out how to import regularizers
from tensorflow.keras.regularizers import l1, l2

###
import tensorflow.keras.utils as ku 
import numpy as np 

In [2]:
tokenizer = Tokenizer()
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
    -O /tmp/sonnets.txt
data = open('/tmp/sonnets.txt').read()

corpus = data.lower().split("\n")


tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)


# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

--2020-03-16 03:00:41--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.14.112, 2607:f8b0:4026:800::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.14.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘/tmp/sonnets.txt’

/tmp/sonnets.txt    100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2020-03-16 03:00:41 (166 MB/s) - ‘/tmp/sonnets.txt’ saved [93578/93578]



In [10]:
inverse_word_index = {value: key for key, value in tokenizer.word_index.items()}

def inverse_seq(seq):
    return ' '.join([inverse_word_index[i] for i in seq if i != 0])

print(corpus[0])
print(input_sequences[3])
print(inverse_seq(predictors[3]))
print(inverse_seq(label[3]))

from fairest creatures we desire increase,
[  0   0   0   0   0   0  34 417 877 166 213]
from fairest creatures we
and


In [15]:
label.shape[1]

3211

In [20]:
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=100, input_length=max_sequence_len-1))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(rate=0.2))
model.add(LSTM(32))
regularizer=Regularizer()
model.add(Dense(1000, activation='relu', kernel_regularizer=l2(0.01),
                bias_regularizer=l1(0.01)))
model.add(Dense(label.shape[1], activation='sigmoid'))
# Pick an optimizer
adam = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
print(model.summary())


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 10, 100)           321100    
_________________________________________________________________
lstm_6 (LSTM)                (None, 10, 64)            42240     
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              33000     
_________________________________________________________________
dense_3 (Dense)              (None, 3211)              3214211   
Total params: 3,622,967
Trainable params: 3,622,967
Non-trainable params: 0
____________________________________________

In [21]:
 history = model.fit(predictors, label, epochs=100, verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 15462 samples
Epoch 1/100
15462/15462 [==============================] - 16s 1ms/sample - loss: 6.9518 - acc: 0.0219
Epoch 2/100
15462/15462 [==============================] - 14s 875us/sample - loss: 6.7845 - acc: 0.0219
Epoch 3/100
15462/15462 [==============================] - 14s 923us/sample - loss: 6.8289 - acc: 0.0224
Epoch 4/100
15462/15462 [==============================] - 15s 944us/sample - loss: 6.8102 - acc: 0.0227
Epoch 5/100
15462/15462 [==============================] - 14s 907us/sample - loss: 6.8194 - acc: 0.0228
Epoch 6/100
15462/15462 [==============================] - 14s 937us/sample - loss: 6.7972 - acc: 0.0233
Epoch 7/100
15462/15462 [==============================] - 14s 924us/sample - loss: 6.7840 - acc: 0.0228
Epoch 8/100
15462/15462 [==============================] - 14s 906us/sample - loss: 6.7644 - acc: 0.0237
Epoch 9/100
15462/15462 [====================

In [0]:
import matplotlib.pyplot as plt
acc = history.history['acc']
loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.title('Training accuracy')

plt.figure()

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()

plt.show()

In [0]:
seed_text = "Help me Obi Wan Kenobi, you're my only hope"
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)